# Covid-19 Dashboard

The following graph compares the total number of Covid-19 cases since the beginning of the pandemic to the total number of hospital admissions associated with Covid-19. 

In [1]:
from IPython.display import clear_output 
from uk_covid19 import Cov19API 
import ipywidgets as wdg 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json 

# sets the visual output of the graph so that it is larger and easier to read
%matplotlib inline 
plt.rcParams['figure.dpi'] = 100

In [2]:
# This block reads the contents of JSON files and assigns them to variable names.

# returns the contents of a JSON file
def contentsOf(fileName: str):
    with open(fileName, "rt") as INFILE:
        return json.load(INFILE)

# returns only the 'data' portion of a JSON file
def rawData(fileName: str):
    data = contentsOf(fileName)
    return data['data']

dataset_one = rawData("newCasesAndNewTests.json")
dataset_two = rawData("ukCasesAndAdmissions.json")

In [3]:
# This block performs the data wrangling necessary to create the graph.

# reads the 'date' value from the data and converts it into optimal format
def parse_date(datestring: str):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# sorts through and reformats the data for use in the graph
def wrangle_data(dataset: str, setColumns: list):
    # sorts data by date
    dates = [dictionary['date'] for dictionary in dataset]
    dates.sort()
    
    # sets start and end dates for the graph
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

    # sets index to date range and creates dataframe
    index = pd.date_range(startdate, enddate, freq='D')
    df = pd.DataFrame(index = index, columns = setColumns)

    # populates columns in the dataset
    for entry in dataset:
        date = parse_date(entry['date'])
        for column in setColumns:
            if pd.isna(df.loc[date, column]):
                value = float(entry[column]) if entry[column]!= None else 0.0
                df.loc[date, column] = value 

    # fills in remaining holes
    df.fillna(0.0, inplace = True)

    return df 

newCasesAndNewTestsdf = wrangle_data(dataset_one, ['new_cases', 'new_tests'])
ukCasesAndAdmissionsdf = wrangle_data(dataset_two, ['cases', 'admissions'])

In [4]:
# This block creates the first graph and adds interactive controls.

def graph_one(gcols: str, gscale: str):
    if gscale == 'linear':
        logscale = False 
    else:
        logscale = True 

    ncols = len(gcols)

    if ncols > 0:
        newCasesAndNewTestsdf[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print('Click to select data for graph')
        print('CTRL-Click to select more than one category') 

# sets values of series (column) selection
series_one = wdg.SelectMultiple(
    options = ['new_cases', 'new_tests'],
    value = ['new_cases', 'new_tests'],
    rows = 2,
    description = 'Statistics:',
    disable = False
)

# sets values of scale selection
scale_one = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:', 
    disabled = False
) 

# defines appearance of the graph controls using series and select variables
controls_one = wdg.HBox([series_one, scale_one])

# creates UI and organizes it within a box
graph_one = wdg.interactive_output(graph_one, {'gcols': series_one, 'gscale': scale_one})

# outputs graph and interactive controls
display(controls_one, graph_one)

Output()

In [5]:
# This block creates a button that allows a user to refresh the data for graph one by polling the API. 

# accesses API to update the dataset
def access_api():
    # sets filter
    united_kingdom = [
        'areaType=overview'
    ]

    # sets structure
    new_cases_and_tests = {
        "date": "date",
        "new_cases": "newCasesByPublishDate",
        "new_tests": "newTestsByPublishDate"
    }

    # creates 'api' object
    api = Cov19API(filters=united_kingdom, structure=new_cases_and_tests)

    # polls the API for data
    data = api.get_json()

    return data['data']

# defines function of the button widget
def api_button_callback(button):
    apidata = access_api()

    global newCasesAndNewTestsdf

    # sets global dataframe variable to a wrangled version of the new dataset
    newCasesAndNewTestsdf = wrangle_data(apidata, ['new_cases', 'new_tests'])

    # sets visual appearance of the button
    apibutton.icon = "check"

apibutton = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Get new data from the API', 
    icon = 'check'
)

apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='info', description='Refresh Data', icon='check', style=ButtonStyle(), tooltip='Get new da…

In [6]:
# This block creates the second graph and adds interactive controls.

def graph_two(gcols: str, gscale: str):
    if gscale == 'linear':
        logscale = False 
    else:
        logscale = True 

    ncols = len(gcols)

    if ncols > 0:
        ukCasesAndAdmissionsdf[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print('Click to select data for graph')
        print('CTRL-Click to select more than one category') 

# sets values of series (column) selection
series_two = wdg.SelectMultiple(
    options = ['cases', 'admissions'],
    value = ['cases', 'admissions'],
    rows = 2,
    description = 'Statistics:',
    disable = False
)

# sets values of scale selection
scale_two = wdg.RadioButtons(
    options = ['linear', 'log'],
    description = 'Scale:', 
    disabled = False
) 

# defines appearance of the graph controls using series and select variables
controls_two = wdg.HBox([series_two, scale_two])

# creates UI and organizes it within a box
graph_two = wdg.interactive_output(graph_two, {'gcols': series_two, 'gscale': scale_two})

# outputs graph and interactive controls
display(controls_two, graph_two)

Output()

In [7]:
# This block creates a button that allows a user to refresh the data for graph two by polling the API. 

# accesses API to update the dataset
def access_api_two():
    # sets filter
    united_kingdom = [
        'areaType=overview'
    ]

    # sets structure
    cases_and_admissions = {
        "date": "date",
        "cases": "cumCasesBySpecimenDate",
        "admissions": "cumAdmissions"
    }

    # creates 'api' object
    api = Cov19API(filters=united_kingdom, structure=cases_and_admissions)

    # polls the API for data
    data = api.get_json()

    return data['data']

# defines function of the button widget
def api_button_callback_two(button):
    apidata = access_api_two()

    global ukCasesAndAdmissionsdf

    # sets global dataframe variable to a wrangled version of the new dataset
    ukCasesAndAdmissionsdf = wrangle_data(apidata, ['cases', 'admissions'])

    # sets visual appearance of the button
    apibutton_two.icon = "check"

apibutton_two = wdg.Button(
    description = 'Refresh Data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Get new data from the API', 
    icon = 'check'
)

apibutton_two.on_click(api_button_callback_two)

display(apibutton_two)

Button(button_style='info', description='Refresh Data', icon='check', style=ButtonStyle(), tooltip='Get new da…